In [ ]:
import pickle

with open('message.pickle', 'wb') as handle:
    pickle.dump(message, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('message.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [11]:
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

engine = create_engine('sqlite:///bot-converter.db', echo=True)
Base = declarative_base()

In [23]:
from sqlalchemy.orm import close_all_sessions

close_all_sessions()

In [13]:
from sqlalchemy import Column, Integer, String
class File(Base):
    __tablename__ = 'files'

    id = Column(Integer, primary_key=True)
    chat_id = Column(Integer)
    filename = Column(String)

    def __repr__(self):
        return "<File(chat_id='{}', filename='{}')>".format(self.chat_id, self.filename)
    
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    chat_id = Column(Integer)
    username = Column(String)
    state = Column(String)

    def __repr__(self):
        return "<User(chat_id='{}', username='{}')>".format(self.chat_id, self.filename)

In [14]:
Base.metadata.create_all(engine)

2022-01-28 13:09:45,718 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-28 13:09:45,720 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("files")
2022-01-28 13:09:45,721 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-28 13:09:45,725 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("files")
2022-01-28 13:09:45,726 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-28 13:09:45,727 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-01-28 13:09:45,728 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-28 13:09:45,729 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-01-28 13:09:45,730 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-28 13:09:45,731 INFO sqlalchemy.engine.Engine 
CREATE TABLE files (
	id INTEGER NOT NULL, 
	chat_id INTEGER, 
	filename VARCHAR, 
	PRIMARY KEY (id)
)


2022-01-28 13:09:45,736 INFO sqlalchemy.engine.Engine [no key 0.00495s] ()
2022-01-28 13:09:45,741 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id IN

In [15]:
ed_file = File(chat_id=0, filename='test.mov')
ed_user = User(chat_id=0, username='admin')

In [16]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [17]:
session.add(ed_file)
session.add(ed_user)

In [18]:
our_file = session.query(File).filter_by(chat_id=0).first() 
print(our_file)

2022-01-28 13:10:18,552 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-28 13:10:18,555 INFO sqlalchemy.engine.Engine INSERT INTO files (chat_id, filename) VALUES (?, ?)
2022-01-28 13:10:18,558 INFO sqlalchemy.engine.Engine [generated in 0.00320s] (0, 'test.mov')
2022-01-28 13:10:18,561 INFO sqlalchemy.engine.Engine INSERT INTO users (chat_id, username, state) VALUES (?, ?, ?)
2022-01-28 13:10:18,564 INFO sqlalchemy.engine.Engine [generated in 0.00316s] (0, 'admin', None)
2022-01-28 13:10:18,567 INFO sqlalchemy.engine.Engine SELECT files.id AS files_id, files.chat_id AS files_chat_id, files.filename AS files_filename 
FROM files 
WHERE files.chat_id = ?
 LIMIT ? OFFSET ?
2022-01-28 13:10:18,572 INFO sqlalchemy.engine.Engine [generated in 0.00540s] (0, 1, 0)
<File(chat_id='0', filename='test.mov')>


In [19]:
session.commit()

2022-01-28 13:10:22,984 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine

engine = create_engine('sqlite:///bot-converter.db', echo=True)
Base = declarative_base()


from sqlalchemy import Column, Integer, String
class File(Base):
    __tablename__ = 'files'

    id = Column(Integer, primary_key=True)
    chat_id = Column(Integer)
    filename = Column(String)

    def __repr__(self):
        return "<File(chat_id='{}', filename='{}')>".format(self.chat_id, self.filename)
    
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    chat_id = Column(Integer)
    username = Column(String)
    state = Column(String)

    def __repr__(self):
        return "<User(chat_id='{}', username='{}')>".format(self.chat_id, self.filename)

from sqlalchemy import create_engine
engine = create_engine('sqlite:///bot-converter.db', echo=True)
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)


import telebot
from telebot import types

bot = telebot.TeleBot("5069674898:AAGdHYM4cn-ZqQqbhR2vke34ke8qMos1lTY", parse_mode=None)

WELCOME_MSG = "👋 Howdy, how are you doing? \
I can convert video to the following formats: *.avi, *.mov, *.mp4! and convert to *.avi, *.mov, *.mp4, *.gif! \
Just upload a short video for me, and then choose file type you want. \
/start, /setname, /help commands are available!"

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, WELCOME_MSG)
    ed_user = User(chat_id=message.chat.id, state='start')
    session = Session()
    session.add(ed_user)
    session.commit()
    session.close()
    
@bot.message_handler(commands=['setname'])
def set_name(message):
    bot.reply_to(message, 'Set your username')
    session = Session()
    our_user = session.query(User).filter_by(chat_id=message.chat.id).order_by(User.id.desc()).first()
    our_user.state = 'set_name'
    session.commit()
    session.close()

@bot.message_handler(content_types=['video', 'video_note', 'document'])
def handle_docs_video(message):
    print(message.content_type)
    # Get file
    if message.content_type == 'video_note':
        file_info = bot.get_file(message.video_note.file_id)
    elif message.content_type == 'video':
        file_info = bot.get_file(message.video.file_id)
    elif message.content_type == 'document':
        file_info = bot.get_file(message.document.file_id)
    
    ## Check if filetype is supported 
    file_type = file_info.file_path.split('.')[-1].lower()
    if (file_type in ['mov', 'mp4', 'avi']):
        bot.reply_to(message, 'Detected file type: "{}". Congrats! It is supported!'.format(file_type))
    else:
        bot.reply_to(message, 'Detected file type: "{}". Sorry is not supported!'.format(file_type))
    
    ## Save temporary file
    downloaded_file = bot.download_file(file_info.file_path)
    temp_filename = str(file_info.file_path).split('/')[-1]
    with open(temp_filename,'wb') as new_file:
        new_file.write(downloaded_file)
        
    # Get target filetype
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('/avi')
    itembtn2 = types.KeyboardButton('/mov')
    itembtn3 = types.KeyboardButton('/mp4')
    itembtn4 = types.KeyboardButton('/gif')
    
    

    markup.add(itembtn1, itembtn2, itembtn3, itembtn4)
    bot.send_message(message.chat.id, "Choose the target file type:", reply_markup=markup)
    
    ed_file = File(chat_id=message.chat.id, filename=temp_filename)
    session = Session()
    session.add(ed_file)
    session.commit()
    session.close()

@bot.message_handler(commands=['avi', 'mov', 'mp4', 'gif'])
def process_file(message):
    print('processing file')
    markup = types.ReplyKeyboardRemove(selective=False)
    target_file_type = message.text[1:]
    bot.send_message(message.chat.id, 'You\'ve chosen .{} format! Processing...'.format(target_file_type), reply_markup=markup)
    
    session = Session()
    our_file = session.query(File).filter_by(chat_id=message.chat.id).order_by(File.id.desc()).first()
    session.close()
    
    temp_filename = our_file.filename
    
    # Convert file
    import ffmpeg
    try:
        stream = ffmpeg.input(temp_filename)
        output_name = '{}.{}'.format(temp_filename.split('.')[0], target_file_type)
        stream = ffmpeg.output(stream, output_name)
        ffmpeg.run(stream)
    except Exception as e:
        print(e)
        bot.send_message(message.chat.id, "Ooops! An error occured! Try again! {}".format(e))
    
    # Send file
    with open(output_name,'rb') as converted_file:
        session = Session()
        our_user = session.query(User).filter_by(chat_id=message.chat.id).order_by(User.id.desc()).first()
        session.close()
        if our_user.username:
            bot.send_message(message.chat.id, 'Thank you for using our service, {}'.format(our_user.username))
        bot.send_document(message.chat.id, converted_file)
    import os
    os.remove(output_name)

@bot.message_handler(func=lambda message: True)
def echo_all(message):
    session = Session()
    our_user = session.query(User).filter_by(chat_id=message.chat.id).order_by(User.id.desc()).first()
    
    if our_user.state == 'set_name':
        our_user.username = message.text
        our_user.state = 'complete'
        session.commit()
        bot.reply_to(message, 'Hello, {}'.format(our_user.username))
        session.close()
    else:
        bot.reply_to(message, WELCOME_MSG)

bot.infinity_polling()

2022-01-28 13:36:30,167 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-28 13:36:30,173 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.chat_id AS users_chat_id, users.username AS users_username, users.state AS users_state 
FROM users 
WHERE users.chat_id = ? ORDER BY users.id DESC
 LIMIT ? OFFSET ?
2022-01-28 13:36:30,176 INFO sqlalchemy.engine.Engine [generated in 0.00313s] (26921314, 1, 0)
2022-01-28 13:36:30,181 INFO sqlalchemy.engine.Engine COMMIT
2022-01-28 13:36:46,449 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-28 13:36:46,453 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.chat_id AS users_chat_id, users.username AS users_username, users.state AS users_state 
FROM users 
WHERE users.chat_id = ? ORDER BY users.id DESC
 LIMIT ? OFFSET ?
2022-01-28 13:36:46,457 INFO sqlalchemy.engine.Engine [cached since 16.28s ago] (26921314, 1, 0)
2022-01-28 13:36:46,461 INFO sqlalchemy.engine.Engine UPDATE users SET username=?, state=? WHERE 

2022-01-28 13:39:31,019 (__init__.py:694 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
2022-01-28 13:39:34,602 (__init__.py:694 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
2022-01-28 13:39:38,648 (__init__.py:694 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
2022-01-28 13:39:41,191 (__init__.py:627 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-01-28 13:39:41,192 (__init__.py:629 MainThread) ERROR - TeleBot: "Break infinity polling"


In [16]:
b.document.file_name

NameError: name 'b' is not defined